In [0]:
!pip install keras-bert

In [2]:
from keras_bert import Tokenizer

token_dict = {
    '[CLS]': 0,
    '[SEP]': 1,
    'un': 2,
    '##aff': 3,
    '##able': 4,
    '[UNK]': 5,
}
tokenizer = Tokenizer(token_dict)
print(tokenizer.tokenize('unaffable'))  # The result should be `['[CLS]', 'un', '##aff', '##able', '[SEP]']`
indices, segments = tokenizer.encode('unaffable')
print(indices)  # Should be `[0, 2, 3, 4, 1]`
print(segments)  # Should be `[0, 0, 0, 0, 0]`

print(tokenizer.tokenize(first='unaffable', second='钢'))
# The result should be `['[CLS]', 'un', '##aff', '##able', '[SEP]', '钢', '[SEP]']`
indices, segments = tokenizer.encode(first='unaffable', second='钢', max_len=10)
print(indices)  # Should be `[0, 2, 3, 4, 1, 5, 1, 0, 0, 0]`
print(segments)  # Should be `[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]`

Using TensorFlow backend.


['[CLS]', 'un', '##aff', '##able', '[SEP]']
[0, 2, 3, 4, 1]
[0, 0, 0, 0, 0]
['[CLS]', 'un', '##aff', '##able', '[SEP]', '钢', '[SEP]']
[0, 2, 3, 4, 1, 5, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


In [0]:
from keras_bert import get_base_dict, get_model, gen_batch_inputs


# A toy input example
sentence_pairs = [
    [['all', 'work', 'and', 'no', 'play'], ['makes', 'jack', 'a', 'dull', 'boy']],
    [['from', 'the', 'day', 'forth'], ['my', 'arm', 'changed']],
    [['and', 'a', 'voice', 'echoed'], ['power', 'give', 'me', 'more', 'power']],
]


# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word


# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
)
model.summary()

def _generator():
    while True:
        yield gen_batch_inputs(
            sentence_pairs,
            token_dict,
            token_list,
            seq_len=20,
            mask_rate=0.3,
            swap_sentence_rate=1.0,
        )

model.fit_generator(
    generator=_generator(),
    steps_per_epoch=1000,
    epochs=100,
    validation_data=_generator(),
    validation_steps=100,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)


# Use the trained model
inputs, output_layer = get_model(  # `output_layer` is the last feature extraction layer (the last transformer)
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
    training=False,   # The input layers and output layer will be returned if `training` is `False`
    trainable=False,  # Whether the model is trainable. The default value is the same with `training`
)

In [0]:
import keras as keras
def _custom_layers(x, trainable=True):
    return keras.layers.LSTM(
        units=768,
        trainable=trainable,
        return_sequences=True,
        name='LSTM',
    )(x)

model = get_model(
    token_num=200,
    embed_dim=768,
    custom_layers=_custom_layers,
)

In [0]:
!pip install pytorch-pretrained-bert

In [7]:
from pytorch_pretrained_bert import BertTokenizer

# Hello! Does this work with German sentences, too?
text = 'Ik snap het niet (informeel)'
text_one = 'Ik weet het niet meer'
text_two = 'Het is duidelijk'
text_three = 'Ik weet niet hoe je dat zegt in het NL'
text_four = "TMG websites en apps maken gebruik van cookies en vergelijkbare technieken i) voor functionele en analytische doeleinden, ii) om het mogelijk te maken content via social media te delen, iii) om informatie te verzamelen over uw voorkeuren en de informatie toe te voegen aan uw klantprofiel en iv) om de inhoud van haar websites en advertenties af te stemmen op uw voorkeuren. Cookies kunnen ook geplaatst en gebruikt worden door derden, zoals advertentienetwerken, adverteerders en technologiepartners. Meer informatie over het cookiegebruik op de TMG websites en apps, waaronder de specifieke cookies, doeleinden en bewaartermijnen vindt u hier. TMG deelt de informatie die zij verkrijgt middels het gebruik van cookies en vergelijkbare technieken, waaronder ook persoonsgegevens,  in een samenwerkingsverband genaamd NLProfiel van Buymedia Nederland met Sanoma en de Persgroep, om gezamenlijke groepsprofielen op te stellen. Ook gaat u dan akkoord met de verwerking van de (persoons)gegevens die met behulp van cookies door TMG en derden kunnen worden verzameld en verwerkt voor de onder i) tot en met iv) genoemde doeleinden, alsmede met de verstrekking van uw (persoons)gegevens aan Sanoma en de Persgroep voor de totstandkoming van de gezamenlijke groepsprofielen. Meer informatie over deze verwerking van (persoons)gegevens kunt u vinden in de privacyverklaring van TMG respectievelijk van deze derden."


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_text = tokenizer.tokenize(text)
tokenized_text_one = tokenizer.tokenize(text_one)
tokenized_text_two = tokenizer.tokenize(text_two)
tokenized_text_three = tokenizer.tokenize(text_three)
tokenized_text_four = tokenizer.tokenize(text_four)


print(tokenized_text)
print(tokenized_text_one)
print(tokenized_text_two)
print(tokenized_text_three)
print(tokenized_text_four)




100%|██████████| 995526/995526 [00:00<00:00, 12554008.90B/s]


['Ik', 'sna', '##p', 'het', 'niet', '(', 'informe', '##el', ')']
['Ik', 'weet', 'het', 'niet', 'meer']
['Het', 'is', 'duidelijk']
['Ik', 'weet', 'niet', 'hoe', 'je', 'dat', 'ze', '##gt', 'in', 'het', 'NL']
['TM', '##G', 'websites', 'en', 'app', '##s', 'maken', 'gebruik', 'van', 'co', '##okie', '##s', 'en', 'ver', '##gelijk', '##bare', 'tech', '##niek', '##en', 'i', ')', 'voor', 'function', '##ele', 'en', 'anal', '##yti', '##sche', 'doel', '##ein', '##den', ',', 'ii', ')', 'om', 'het', 'mogelijk', 'te', 'maken', 'content', 'via', 'social', 'media', 'te', 'delen', ',', 'iii', ')', 'om', 'informatie', 'te', 'ver', '##zame', '##len', 'over', 'u', '##w', 'voor', '##keur', '##en', 'en', 'de', 'informatie', 'toe', 'te', 'voe', '##gen', 'aan', 'u', '##w', 'kl', '##ant', '##pro', '##fiel', 'en', 'i', '##v', ')', 'om', 'de', 'in', '##houd', 'van', 'haar', 'websites', 'en', 'ad', '##verte', '##nties', 'af', 'te', 'stemmen', 'op', 'u', '##w', 'voor', '##keur', '##en', '.', 'Cook', '##ies', 'kunnen

In [8]:
from pytorch_pretrained_bert import BasicTokenizer, BertTokenizer

# Insert example text
text = '...'

basic_tokenizer = BasicTokenizer(do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

in_vocab = []
not_in_vocab = []

for word in basic_tokenizer.tokenize(text):
    if word in tokenizer.vocab:
        in_vocab.append(word)
    else:
        not_in_vocab.append(word)

print(len(in_vocab))
print(len(not_in_vocab))

3
0
